# Preliminaries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
#from feature_selection import forwardFeatureSelection

sys.path.append("../LinCFA")
from LinCFA import LinCFA

sys.path.append("../NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("../GenLinCFA")
from GenLinCFA import GenLinCFA

sys.path.append("../droughts")
from aux import prepare_target,prepare_features,compare_methods

#from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2


In [2]:
df = pd.read_csv('droughts_extended.csv')
df_trainVal_withTar = df.iloc[:-392,:]
df_test_withTar = df.iloc[-392:,:]
df_trainVal = df.iloc[:-392,:-1]
df_test = df.iloc[-392:,:-1]
target_df_trainVal = df.iloc[:-392,-1]
target_df_test = df.iloc[-392:,-1]

In [11]:
def compute_lower_fromList(lis, list_len, n_reps):
    res = []
    for i in range(list_len):
        res.append(np.mean(lis[i::list_len])-1.96*np.std(lis[i::list_len])/np.sqrt(n_reps))
    return res

def compute_upper_fromList(lis, list_len, n_reps):
    res = []
    for i in range(list_len):
        res.append(np.mean(lis[i::list_len])+1.96*np.std(lis[i::list_len])/np.sqrt(n_reps))
    return res

def compute_means_fromList(lis, list_len):
    res = []
    for i in range(list_len):
        res.append(np.mean(lis[i::list_len]))
    return res

def compute_CI(list,n):
    print(f'{np.mean(list)} +- {1.96*np.std(list)/np.sqrt(n)}')

# Droughts

# GenLinCFA

In [3]:
#for variable in ['cyclostationary_mean_tg']:#variables_list:
#actual_df_trainVal = df_trainVal_unfolded_std[df_trainVal_unfolded_std.columns[pd.Series(df_trainVal_unfolded_std.columns).str.startswith(variable)]]
for curr_seed in [0,1,2,3]:
    for eps in [0.365,0.37,0.375,0.38,0.39]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        output = GenLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0, 1).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        compare_methods(aggregate_trainVal, aggregate_test, df_trainVal_withTar, df_test_withTar, list(aggregate_trainVal.columns))


991
976
1
1
1
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 23

Full aggregate regression train score: 0.9209950699181899, test score: 0.9159749397150054
Aggregate regression train score with FS: 0.9209950699181899, test score: 0.9159749397150054
991
986
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 16

Full aggregate regression train score: 0.9163948059371616, test score: 0.9235972056959854
Aggregate regression train score with FS: 0.9163948059371616, test score: 0.9235972056959854
991
991
1
1
1
1
1
1
1
1
1
Number of aggregated features: 11

Full aggregate regression train score: 0.9119383771719635, test score: 0.9283816292134218
Aggregate regression train score with FS: 0.9119383771719635, test score: 0.9283816292134218
991
991
1
1
1
1
1
1
1
1
1
Number of aggregated features: 11

Full aggregate regression train score: 0.9119383771719635, test score: 0.9283816292134218
Aggregate regression train score with FS: 0.9119383771719635, test score: 0.9283816

In [4]:
for curr_seed in [0,1,2,3]:
    for eps in [0.36,0.362,0.364, 0.366, 0.368]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        output = GenLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0, 1).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        compare_methods(aggregate_trainVal, aggregate_test, df_trainVal_withTar, df_test_withTar, list(aggregate_trainVal.columns))


991
946
5
1
12
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 41

Full aggregate regression train score: 0.9268491384275956, test score: 0.9156773980796991
Aggregate regression train score with FS: 0.9268491384275956, test score: 0.9156773980796991
991
959
2
9
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 34

Full aggregate regression train score: 0.9258344676273087, test score: 0.9190625275082789
Aggregate regression train score with FS: 0.9258344676273087, test score: 0.9190625275082789
991
972
1
7
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 24

Full aggregate regression train score: 0.9209800985391119, test score: 0.9157822753027478
Aggregate regression train score with FS: 0.9209800985391119, test score: 0.9157822753027478
991
981
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 21

Full aggregate regression train score: 0.9177080763666

In [10]:
r2_036 = [0.9156773980796991,0.9279180559917267,0.917201303885054,0.920072798775049,0.9159749397150054]
r2_0362 = [0.9190625275082789,0.9279187378765488,0.9214827231165493,0.9213167806257397,0.9235972056959854]
r2_0364 = [0.9157822753027478,0.9278726678162809,0.9160898658275374,0.9157941792685421,0.9283816292134218]
r2_0366 = [0.925160660166235,0.9278668053556873,0.9252118016513883,0.9282411785227777,0.9283816292134218]
r2_0368 = [0.927014786172269,0.9278565928836973,0.9270514575504571,0.9282335108375251,0.927656834295981]

n_036 = [41,11,42,41,23]
n_0362 = [34,11,34,35,16]
n_0364 = [24,11,23,24,11]
n_0366 = [21,11,21,11,11]
n_0368 = [18,11,18,11,11]

compute_CI(r2_036,5)
compute_CI(r2_0362,5)
compute_CI(r2_0364,5)
compute_CI(r2_0366,5)
compute_CI(r2_0368,5)

compute_CI(n_036,5)
compute_CI(n_0362,5)
compute_CI(n_0364,5)
compute_CI(n_0366,5)
compute_CI(n_0368,5)


0.9193688992893069 +- 0.003986625881376614
0.9226755949646204 +- 0.002619765358266197
0.920784123485706 +- 0.005258116186787761
0.9269724149819021 +- 0.001286918425275126
0.9275626363479859 +- 0.0004124018589180096
31.6 +- 10.970398570699244
26.0 +- 9.058508486500411
18.6 +- 5.448658990981175
15.0 +- 4.294144850840501
13.8 +- 3.005901395588351


# LinCFA

In [5]:
for curr_seed in [0,1,2,3]:
    curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
    curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
    curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
    
    output = LinCFA(curr_df_trainVal_withTar,'mean_std', 0, 0).compute_clusters()
    aggregate_trainVal = pd.DataFrame()
    aggregate_test = pd.DataFrame()
    
    for i in range(len(output)):
        aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
        aggregate_trainVal = aggregate_trainVal.copy()
        aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
        aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        compare_methods(aggregate_trainVal, aggregate_test, df_trainVal_withTar, df_test_withTar, list(aggregate_trainVal.columns))


Number of aggregated features: 40

Full aggregate regression train score: 0.09553760440374637, test score: 0.07197922433984683
Aggregate regression train score with FS: 0.09553760440374637, test score: 0.07197922433984683
Number of aggregated features: 40

Full aggregate regression train score: 0.1711788819597201, test score: 0.13154881611228453
Aggregate regression train score with FS: 0.1711788819597201, test score: 0.13154881611228453
Number of aggregated features: 40

Full aggregate regression train score: 0.17930536071547898, test score: 0.14334294886686194
Aggregate regression train score with FS: 0.17930536071547898, test score: 0.14334294886686194
Number of aggregated features: 40

Full aggregate regression train score: 0.18433794519276503, test score: 0.1321139735994563
Aggregate regression train score with FS: 0.18433794519276503, test score: 0.1321139735994563
Number of aggregated features: 40

Full aggregate regression train score: 0.19900134252850576, test score: 0.1144872

Number of aggregated features: 36

Full aggregate regression train score: 0.0873718410661879, test score: 0.07097850058564226
Aggregate regression train score with FS: 0.0873718410661879, test score: 0.07097850058564226
Number of aggregated features: 36

Full aggregate regression train score: 0.17157143913953443, test score: 0.1357624317491789
Aggregate regression train score with FS: 0.17157143913953443, test score: 0.1357624317491789
Number of aggregated features: 36

Full aggregate regression train score: 0.18257892180684032, test score: 0.13986879066234037
Aggregate regression train score with FS: 0.18257892180684032, test score: 0.13986879066234037
Number of aggregated features: 36

Full aggregate regression train score: 0.18413791518540423, test score: 0.14807935019289198
Aggregate regression train score with FS: 0.18413791518540423, test score: 0.14807935019289198
Number of aggregated features: 36

Full aggregate regression train score: 0.18563137983544442, test score: 0.1408371

Number of aggregated features: 40

Full aggregate regression train score: 0.09671550062026057, test score: 0.07575804997490809
Aggregate regression train score with FS: 0.09671550062026057, test score: 0.07575804997490809
Number of aggregated features: 40

Full aggregate regression train score: 0.17204375190610632, test score: 0.13011466714104214
Aggregate regression train score with FS: 0.17204375190610632, test score: 0.13011466714104214
Number of aggregated features: 40

Full aggregate regression train score: 0.17329112307944505, test score: 0.12658072931802322
Aggregate regression train score with FS: 0.17329112307944505, test score: 0.12658072931802322
Number of aggregated features: 40

Full aggregate regression train score: 0.18506526735808349, test score: 0.13665079217232756
Aggregate regression train score with FS: 0.18506526735808349, test score: 0.13665079217232756
Number of aggregated features: 40

Full aggregate regression train score: 0.1874168865281245, test score: 0.1210

In [11]:
r2_Lin = [40,36,37,40]
n_Lin = [0.9319920849520081,0.842567646364767,0.9244750451206315,0.9039566279930502]

compute_CI(r2_Lin,5)

compute_CI(n_Lin,5)

38.25 +- 1.5649345034217887
0.9007478511076141 +- 0.030786398128805483


# NonLinCFA

In [6]:
import pickle
with open('../results/droughts_NonLinCFA_new.pkl', 'rb') as f:
    nonLinCFA_res = pickle.load(f)

nonLinCFA_res

[[0.01, 0, 8, 0.8816038213788577],
 [0.01, 1, 6, 0.7921001741839128],
 [0.01, 2, 7, 0.8684521184935183],
 [0.01, 3, 10, 0.9145019501433567],
 [0.01, 4, 6, 0.8058315228852355],
 [0.001, 0, 16, 0.9464198373237047],
 [0.001, 1, 17, 0.9410284393800941],
 [0.001, 2, 17, 0.9118203997740284],
 [0.001, 3, 15, 0.9511715200041537],
 [0.001, 4, 15, 0.9469936536731554],
 [0.0001, 0, 17, 0.9127408301616642],
 [0.0001, 1, 18, 0.9095386127349159],
 [0.0001, 2, 21, 0.9157372349521051],
 [0.0001, 3, 21, 0.9213932119871185],
 [0.0001, 4, 20, 0.9024647999808804],
 [1e-05, 0, 19, 0.9125905571887877],
 [1e-05, 1, 20, 0.9054267356676863],
 [1e-05, 2, 18, 0.914713249352745],
 [1e-05, 3, 21, 0.9205771871747598],
 [1e-05, 4, 20, 0.9030394281207829],
 [1e-06, 0, 18, 0.9120562382036198],
 [1e-06, 1, 20, 0.9112449554321718],
 [1e-06, 2, 18, 0.9147236352713851],
 [1e-06, 3, 22, 0.919396038908035],
 [1e-06, 4, 20, 0.903039428120783]]

In [12]:
r2_01 = [0.8816038213788577, 0.7921001741839128, 0.8684521184935183, 0.9145019501433567, 0.8058315228852355]
r2_001 = [0.9464198373237047, 0.9410284393800941, 0.9118203997740284, 0.9511715200041537, 0.9469936536731554]
r2_0001 = [0.9127408301616642, 0.9095386127349159, 0.9157372349521051, 0.9213932119871185, 0.9024647999808804]
r2_00001 = [0.9125905571887877, 0.9054267356676863, 0.914713249352745, 0.9205771871747598, 0.9030394281207829]
r2_000001 = [0.9120562382036198, 0.9112449554321718, 0.9147236352713851, 0.919396038908035, 0.903039428120783]

n_01 = [8,6,7,10,6]
n_001 = [16,17,17,15,15]
n_0001 = [17,18,21,21,20]
n_00001 = [19,20,18,21,20]
n_000001 = [18,20,18,22,20]

compute_CI(r2_01,5)
compute_CI(r2_001,5)
compute_CI(r2_0001,5)
compute_CI(r2_00001,5)
compute_CI(r2_000001,5)

compute_CI(n_01,5)
compute_CI(n_001,5)
compute_CI(n_0001,5)
compute_CI(n_00001,5)
compute_CI(n_000001,5)

0.8524979174169761 +- 0.040684671411801904
0.9394867700310273 +- 0.01245029825459792
0.9123749379633368 +- 0.0055301706192929095
0.9112694315009524 +- 0.005572878314587403
0.9120920591871989 +- 0.004686692550254596
7.4 +- 1.3118829216054302
16.0 +- 0.7839999999999999
19.4 +- 1.4242067265674598
19.6 +- 0.8938975332777241
19.6 +- 1.3118829216054304


# GenLinCFA classification

In [2]:
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('droughts_extended.csv')
df['mean_std'] = df.apply(lambda x: np.sign(x.mean_std),axis=1)
df_trainVal_withTar = df.iloc[:-392,:]
df_test_withTar = df.iloc[-392:,:]
df_trainVal = df.iloc[:-392,:-1]
df_test = df.iloc[-392:,:-1]
target_df_trainVal = df.iloc[:-392,-1]
target_df_test = df.iloc[-392:,-1]

In [7]:
for curr_seed in [0,1,2,3]:
    for eps in [0.32]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        output = GenLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0, 1/2).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        mod = LogisticRegression(random_state=0).fit(aggregate_trainVal, df_trainVal_withTar.mean_std)
        print(mod.score(aggregate_test, df_test_withTar.mean_std))

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403


22
23
24
25
26
27
28
29
30
31
32
33
33
1
1
1
1
1
1
1
Number of aggregated features: 11

0.9132653061224489
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251

125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374


In [8]:
for curr_seed in [0,1,2,3]:
    for eps in [0.322,0.324,0.326,0.328]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        output = GenLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0, 1/2).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        mod = LogisticRegression(random_state=0).fit(aggregate_trainVal, df_trainVal_withTar.mean_std)
        print(mod.score(aggregate_test, df_test_withTar.mean_std))

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395


259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508


371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620


511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760


...

319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568


429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678


545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794


658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907


771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
985
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [12]:
r2_032 = [0.9030612244897959,0.9132653061224489,0.9030612244897959,0.9081632653061225]
r2_0322 = [0.9056122448979592,0.9132653061224489,0.9056122448979592,0.9056122448979592]
r2_0324 = [0.9030612244897959,0.9132653061224489,0.9030612244897959,0.9030612244897959]
r2_0326 = [0.9056122448979592,0.9132653061224489,0.9030612244897959,0.9030612244897959]
r2_0328 = [0.9081632653061225,0.9132653061224489,0.9081632653061225,0.9132653061224489]

n_032 = [41,11,42,41]
n_0322 = [33,11,33,31]
n_0324 = [26,11,26,26]
n_0326 = [24,11,25,25]
n_0328 = [20,11,20,19]

compute_CI(r2_032,5)
compute_CI(r2_0322,5)
compute_CI(r2_0324,5)
compute_CI(r2_0326,5)
compute_CI(r2_0328,5)

compute_CI(n_032,5)
compute_CI(n_0322,5)
compute_CI(n_0324,5)
compute_CI(n_0326,5)
compute_CI(n_0328,5)

0.9068877551020408 +- 0.003708099243547839
0.9075255102040817 +- 0.002904737509655534
0.9056122448979591 +- 0.003872983346207421
0.9062499999999999 +- 0.0036657195746537963
0.9107142857142858 +- 0.0022360679774997673
33.75 +- 11.518648358205922
27.0 +- 8.128685010504466
22.25 +- 5.6932855189249025
21.25 +- 5.199544210793865
17.5 +- 3.30886083116229
